# 正确重载运算符

## Python对重载的性质
- 不能重置内置类型的运算符
- 不能新建运算符，只能重载现有的
- 某些运算符不能重载——is、and、or和not

## 重载一元运算符
- ```__neg__```重载取反，-
- ```__pos__```重载一元取正运算符，+
- ```__invert__```重载按位取反，~
- 重载之后始终返回一个新对象

## 重载向量加法运算符
- 要求：对应分量相加，返回的依然是向量类型，而且长短不一时将短向量补0之后再相加
- 除非增量运算，否则重载操作一般不会修改操作数

In [1]:
import itertools

In [ ]:
def __add__(self, other):
    pairs = itertools.zip_longest(self, other, fillvalue=0.0)
    return Vector(a + b for a, b in pairs)

## 上面实现的方法是左操作符
- 假如左操作数是Vector之外的类型，就会报错
- 对a+b来说，Python解释器的解释顺序
    - a重载了add方法，就用```a.__add__(b)```
    - a没有，如果b反向/反射/右向实现版本，就用```b.__radd__(a)```
    - b也没有，就抛出TypeError
- 可以直接委托add方法来实现radd

In [ ]:
def __radd__(self, other):
    return self + other

## 为了安全重载运算符
- 我们应该捕获并抛出异常

In [ ]:
def __add__(self, other):
    try:
        pairs = itertools.zip_longest(self, other, fillvalue=0.0)
        return Vector(a + b for a, b in pairs)
    except TypeError:
        return NotImplemented


def __radd__(self, other):
    return self + other

## 另一次安全重载
- 这次我们重载向量与标量的乘积，也是为了捕获异常，但是不采用TypeError这种鸭子类型的方式（判断结果类型）
- 而是使用isinstance这种白鹅类型的方式（判断操作数）

In [ ]:
def __mul__(self, scalar):
    if isinstance(scalar, numbers.Real):
        return Vector(n * scalar for n in self)
    else:
        return NotImplemented


def __rmul__(self, scalar):
    return self * scalar

## 前缀
- 反向方法加r
- 就地方法加i

## 比较运算符
- 对于比较运算符，正向和反向调用的是同一系列方法（只是参数对调）
- 如果调用左操作数的比较返回未实现提示之后，Python会再调用右操作数的比较方法
- 如果还是返回未实现，则Python会最后尝试比较两个对象的id

## 增量运算符
- 如果没有特意实现，则Python的增量运算符只是语法糖，会创建一个新的实例

In [2]:
a = 5
id(5)

140732103242704

In [3]:
b = a
id(b)

140732103242704

In [4]:
a += 1
id(a)

140732103242736

In [6]:
id(b)

140732103242704

## 实现一个可以就地增加的类
- 继承之前我们实现的BingCage
- 同时用isinstance和TypeError来尽可能处理增量运算，并在无法增量运算时给出详细信息
- 如果没有异常，或者不是调用自身（自己实现）的load方法
- 必须返回self

In [11]:
import abc


class Tombola(abc.ABC):
    @abc.abstractmethod
    def load(self, iterable):
        """Add items from an iterable."""

    @abc.abstractmethod
    def pick(self):
        """Remove item at random, returning it.
        This method should raise `LookupError` when the instance is empty.
        """

    def loaded(self):
        """Return `True` if there's at least 1 item, `False` otherwise."""
        return bool(self.inspect())

    def inspect(self):
        """Return a sorted tuple with the items currently inside."""
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))

In [12]:
import random


class BingoCage(Tombola):
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)

    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)

    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):
        self.pick()

In [14]:
import itertools


class AddableBingoCage(BingoCage):
    def __add__(self, other):
        if isinstance(other, Tombola):
            return AddableBingoCage(self.inspect() + other.inspect())
        else:
            return NotImplemented

    def __iadd__(self, other):
        if isinstance(other, Tombola):
            other_iterable = other.inspect()
        else:
            try:
                other_iterable = iter(other)
            except TypeError:
                self_cls = type(self).__name__
                msg = "right operand in += must be {!r} or an iterable"
                raise TypeError(msg.format(self_cls))
        self.load(other_iterable)
        return self